# This notebook launches the LatissAcquireAndTakeSequence SAL script,
### This is used to perform a single target observation as part of the sept 09, 2021 observing run. This is for obs #3 as described in the Spectroscopy-sep2021 document that was used to plan the 2021-09-07 AuxTel run

#### This script is used to both acquire a target and put it within a radius of a pixel that is specific to the grating. The script is then be used to take a series of images with different filter and grating combinations. The focus and pointing adjustments associted with each filter and grating as performed automatically by the ATAOS (so long as the atspectrograph correction is enabled)

In [ ]:
import sys
import os
import asyncio
import time

import numpy as np
import logging 
import yaml
import astropy

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

In [ ]:
# Derive and index for the script that is specific to your user
print(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
print(f'The generated index is {index}')

In [ ]:
# Instantiate the script then start all remotes
script = LatissAcquireAndTakeSequence(index=index)
await script.start_task

In [ ]:
# set the script to have a DEBUG log level
script.log.level = logging.DEBUG

## Declare the Target

In [ ]:
target='HD 185975'
# naively planning to use sequence between HD185975 (pole),  HD200654, HD 009051, HD031128  - BUT THIS MAY CHANGE

In [ ]:
# Print the available instrument filter and gratings
# this is useful both for slewing and for the configuration steps below
inst_setup = await script.latiss.get_available_instrument_setup()
print(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

## Procedure 1 - Performing the acquisition ONLY

In [ ]:
# The configurable parameters can be seen from either inside the script or from uncommenting the following command
# script.get_schema()

In [ ]:
# Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking.
# The position of the target on the detector is related to the grating.
configuration = yaml.safe_dump({"object_name": target,
                                "do_acquire": True,
                                "acq_filter": 'FELH0600',
                                "acq_grating": 'ronchi170lpmm',
                                "acq_exposure_time": 2.0,
                                "target_pointing_tolerance": 3.0,       # Target positioning tolerance (arcsec)
                                "max_acq_iter": 3,                     # Maximum number of iterations to perform when acquiring
                                "target_pointing_verification": True,  # Takes an additional image showing the final target position
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                               })
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

await script.arun()

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()

## Performing the science data collection

In [ ]:
# The configurable parameters can be seen from either inside the script or from uncommenting the following command
# script.get_schema()

In [ ]:
# Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking.
# The position of the target on the detector is related to the grating.
configuration = yaml.safe_dump({"object_name": target,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [20, 20, 20, 20, 10]*2, 
                                "grating_sequence": ['ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                     'ronchi170lpmm',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',
                                                    'holo4_003',],
                                "filter_sequence": ['empty_1','empty_1', 'BG40','FELH0600',
                                                    'quadnotch1',
                                                    'empty_1','empty_1', 'BG40','FELH0600',
                                                    'quadnotch1',], 
                                })
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# run script, assigning the same group ID to all the images taken in the acquisition
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

await script.arun()

In [ ]:
# If required, then stop the telescope from tracking, but then you will lose your acquisition
# await script.atcs.stop_tracking()